In [1]:
!pip install nltk
!pip install tensorflow

In [3]:
import nltk
from nltk.chat.util import Chat, reflections
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
nltk.download('punkt')

# Define patterns and responses for the NLTK chatbot
pairs = [
    ["hi|hello|hey", ["Hello!", "Hi there!", "How can I help you today?"]],
    ["how are you", ["I'm good, thanks. How about you?", "I'm a chatbot, I don't have feelings, but thanks for asking!"]],
    ["what is your name", ["You can call me ChatGPT.", "I'm a chatbot, no specific name."]],
    ["quit|exit", ["Goodbye!", "It was nice chatting with you. Have a great day!"]],
]

# Create the NLTK chatbot
chatbot = Chat(pairs, reflections)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
import requests

# Replace 'your_actual_app_id' and 'your_actual_app_key' with your Edamam API credentials
EDAMAM_APP_ID = 'f6d73819'
EDAMAM_APP_KEY = '188c7ee822658b7f095a211b78634c75'



def get_nutritional_analysis(food_text):
    # Construct the API URL
    api_url = f'https://api.edamam.com/api/nutrition-data?app_id={EDAMAM_APP_ID}&app_key={EDAMAM_APP_KEY}&ingr={food_text}'

    # Make the API request
    response = requests.get(api_url)

    # Handle the API response
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def is_food_healthy(nutritional_data):
    # Check if the food has any health labels indicating it is healthy
    health_labels = nutritional_data.get('healthLabels', [])
    return any(label.lower() in {'balanced', 'high-fiber', 'high-protein', 'low-carb', 'low-fat'} for label in health_labels)

def get_healthy_alternative(food_input):
    # Check if there's a predefined healthy alternative
    alternative = HEALTHY_ALTERNATIVES.get(food_input.lower())
    if alternative:
        return alternative
    else:
        # Query the Edamam API for alternative suggestions
        api_url = f'https://api.edamam.com/api/food-database/v2/parser?app_id={EDAMAM_APP_ID}&app_key={EDAMAM_APP_KEY}&ingr={food_input}'
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            if data.get('hints'):
                return data['hints'][0]['food']['label']
        return None

while True:
    # Take user input for the food item
    food_input = input("Enter the name of the food (type 'quit' to exit): ")

    if food_input.lower() == 'quit':
        print("Exiting the program. Goodbye!")
        break

    # Get nutritional analysis for the user-input food
    nutritional_data = get_nutritional_analysis(food_input)

    if nutritional_data:
        # Extract relevant information from the 'nutritional_data' variable
        print(f"\nNutritional Analysis for {food_input.capitalize()}:")
        calories_data = nutritional_data.get('calories', {})
        if isinstance(calories_data, dict):
            print(f"Calories: {calories_data.get('quantity')} {calories_data.get('unit')}")
        else:
            print(f"Calories: {calories_data} kcal")  # Handle the case when 'calories' is an integer

        print("Nutrients:")
        for nutrient in nutritional_data.get('totalNutrients', []):
            print(f"  {nutrient['label']}: {nutrient['quantity']} {nutrient['unit']}")

        # Check if the food is considered healthy
        if is_food_healthy(nutritional_data):
            print("\nThis food is considered healthy!")
        else:
            print("\nThis food may not be considered very healthy.")
            # Suggest a healthy alternative
            alternative = get_healthy_alternative(food_input)
            if alternative:
                print(f"Consider trying: {alternative.capitalize()} instead.")
            else:
                print("Unfortunately, we don't have a specific suggestion for a healthier alternative.")

        print("\nHealth Labels:", nutritional_data.get('healthLabels', []))
        print("Diet Labels:", nutritional_data.get('dietLabels', []))
    else:
        print(f"Could not retrieve nutritional analysis for {food_input}. Please try again.")


Enter the name of the food (type 'quit' to exit): pizza

Nutritional Analysis for Pizza:
Calories: 0 kcal
Nutrients:

This food may not be considered very healthy.
Consider trying: Grilled chicken salad instead.

Health Labels: ['VEGETARIAN', 'PESCATARIAN', 'EGG_FREE', 'PEANUT_FREE', 'TREE_NUT_FREE', 'SOY_FREE', 'FISH_FREE', 'SHELLFISH_FREE', 'PORK_FREE', 'RED_MEAT_FREE', 'CRUSTACEAN_FREE', 'CELERY_FREE', 'MUSTARD_FREE', 'SESAME_FREE', 'LUPINE_FREE', 'MOLLUSK_FREE', 'ALCOHOL_FREE', 'NO_SUGAR_ADDED', 'KOSHER']
Diet Labels: []
Enter the name of the food (type 'quit' to exit): roti

Nutritional Analysis for Roti:
Calories: 0 kcal
Nutrients:

This food may not be considered very healthy.
Unfortunately, we don't have a specific suggestion for a healthier alternative.

Health Labels: ['VEGAN', 'VEGETARIAN', 'PESCATARIAN', 'MEDITERRANEAN', 'DAIRY_FREE', 'EGG_FREE', 'MILK_FREE', 'PEANUT_FREE', 'TREE_NUT_FREE', 'SOY_FREE', 'FISH_FREE', 'SHELLFISH_FREE', 'PORK_FREE', 'RED_MEAT_FREE', 'CRUSTACEAN